# 02.Entrenamiento del modelo


In [1]:
pip install spacy nltk

     ---------------------------------------- 12.2/12.2 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 1.4 MB/s eta 0:00:00
     ------------------------------------ 122.3/122.3 kB 377.4 kB/s eta 0:00:00
     ---------------------------------------- 1.8/1.8 MB 1.1 MB/s eta 0:00:00
     -------------------------------------- 632.6/632.6 kB 1.5 MB/s eta 0:00:00
     ---------------------------------------- 50.3/50.3 kB 1.3 MB/s eta 0:00:00
     -------------------------------------- 183.0/183.0 kB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 5.4/5.4 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 6.2/6.2 MB 1.1 MB/s eta 0:00:00
     ---------------------------------------- 52.7/52.7 kB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 61.7/61.7 kB 1.1 MB/s eta 0:00:00
     -------------------------------------- 152.0/152.0 kB 1.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use 


[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python -m spacy download es_core_news_sm


     ---------------------------------------- 12.9/12.9 MB 1.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')



[notice] A new release of pip available: 22.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import sqlite3
import datetime
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

nltk.download('punkt')
nltk.download('stopwords')
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SANTI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SANTI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Importacion de la Data

In [5]:


# Guardar en un dataframe
def save_to_dataframe(db_path, table_name):
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect(db_path)
    
    # Leer la tabla en un DataFrame de pandas
    df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
    
    # Cerrar la conexión
    conn.close()
    
    return df




### Noticias

In [6]:
df_news=save_to_dataframe('../Datos/data.db', 'news')
df_news.head(10)

,fecha,fuente,titulo,descripcion,url
0,01/05/2025,Cointelegraph,"CZ: Para tener libertad económica, hay que ten...",El cofundador de Binance dijo que la decisión ...,https://es.cointelegraph.com/news/to-have-free...
1,01/05/2025,Cointelegraph,Bitcoin se recuperó tras datos bajistas del PI...,Los alcistas de Bitcoin regresan con fuerza a ...,https://es.cointelegraph.com/news/bitcoin-rebo...
2,01/05/2025,Expansion.com,Qué siete valores del Ibex han marcado récord ...,"Aena, Endesa, Iberdrola, Indra, Logista, Mapfr...",https://www.expansion.com/mercados/2025/05/01/...
3,01/05/2025,La Nacion,Una primera vuelta en medio de la furia,En medio de un proceso electoral que conmueve ...,https://www.lanacion.com.ar/politica/una-prime...
4,01/05/2025,Cointelegraph,Tether reporta 1.000 millones de dólares en ga...,El exceso actual de reservas de Tether está en...,https://es.cointelegraph.com/news/tether-1b-q1...
5,01/05/2025,Cointelegraph,"La historia del ""desacoplamiento"" de las cript...","Las acciones suben junto a Bitcoin, cuestionan...",https://es.cointelegraph.com/news/crypto-decou...
6,30/04/2025,Cointelegraph,3 gráficos de Ethereum mostraron una señal que...,"Un patrón de velas alcistas, una línea de tend...",https://es.cointelegraph.com/news/3-ethereum-c...
7,30/04/2025,Cointelegraph,Stablecoins alcanzarían los 2 billones de dóla...,La adopción de monedas estables podría acelera...,https://es.cointelegraph.com/news/stablecoins-...
8,30/04/2025,BeInCrypto,El PIB de EEUU alimenta preocupaciones de rece...,A pesar de los preocupantes informes económico...,https://es.beincrypto.com/pib-eeuu-preocupacio...
9,29/04/2025,Expansion.com,"Trump, del júbilo al abismo",Tal vez es la idea que mejor describe la (re)l...,https://www.expansion.com/opinion/2025/04/30/6...


### Historico

In [7]:
df_hist=save_to_dataframe('../Datos/data.db', 'historic')
df_hist.head(100)

,Date,Close_COP=X,High_COP=X,Low_COP=X,Open_COP=X,SMA_30
0,2020-05-04 00:00:00,3954.489990,3954.540039,3912.419189,3954.489990,NaN
1,2020-05-05 00:00:00,3986.909912,3990.000000,3986.810059,3986.909912,NaN
2,2020-05-06 00:00:00,3925.750000,3959.209961,3925.000000,3925.750000,NaN
3,2020-05-07 00:00:00,3961.520020,3963.000000,3914.760010,3961.520020,NaN
4,2020-05-08 00:00:00,3919.090088,3919.379883,3917.000000,3919.090088,NaN
...,...,...,...,...,...,...
95,2020-09-14 00:00:00,3703.760010,3720.280029,3663.654053,3703.760010,3758.382332
96,2020-09-15 00:00:00,3686.919922,3690.159912,3668.550049,3686.919922,3755.667993
97,2020-09-16 00:00:00,3698.360107,3717.449951,3668.127686,3698.360107,3752.938330
98,2020-09-17 00:00:00,3684.250000,3714.169922,3684.250000,3684.250000,3749.954997


# Analisis de sentimientos

In [8]:
df_sentiment=df_news[['fecha','titulo']]
df_sentiment.head()

,fecha,titulo
0,01/05/2025,"CZ: Para tener libertad económica, hay que ten..."
1,01/05/2025,Bitcoin se recuperó tras datos bajistas del PI...
2,01/05/2025,Qué siete valores del Ibex han marcado récord ...
3,01/05/2025,Una primera vuelta en medio de la furia
4,01/05/2025,Tether reporta 1.000 millones de dólares en ga...


In [9]:
stop_words = set(stopwords.words('spanish'))
stemmer = SnowballStemmer('spanish')
def preprocess_text(texto):
    if not isinstance(texto, str):
        return ""
    texto = texto.lower()
    doc = nlp(texto)
    tokens = [token.text for token in doc if token.is_alpha]
    tokens = [t for t in tokens if t not in stop_words]
    tokens = [stemmer.stem(t) for t in tokens]
    return ' '.join(tokens)

In [10]:
df_sentiment['texto_procesado'] = df_sentiment['titulo'].apply(preprocess_text)

C:\Users\SANTI\AppData\Local\Temp\ipykernel_14436\3133770228.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['texto_procesado'] = df_sentiment['titulo'].apply(preprocess_text)


In [11]:
df_sentiment.head()

,fecha,titulo,texto_procesado
0,01/05/2025,"CZ: Para tener libertad económica, hay que ten...",cz ten libert econom ten libert expresion
1,01/05/2025,Bitcoin se recuperó tras datos bajistas del PI...,bitcoin recuper tras dat bajist pib eeuu mient...
2,01/05/2025,Qué siete valores del Ibex han marcado récord ...,siet valor ibex marc record abril
3,01/05/2025,Una primera vuelta en medio de la furia,primer vuelt medi furi
4,01/05/2025,Tether reporta 1.000 millones de dólares en ga...,teth report millon dolar gananci oper prim tri...


In [12]:
# Implementacion transformers para sentimiento
from transformers import pipeline

sentiment_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")


# Aplicar análisis de sentimiento y guardar los resultados
resultados = df_sentiment["texto_procesado"].apply(lambda x: sentiment_pipeline(x)[0])

c:\Users\SANTI\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\SANTI\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SANTI\.cache\huggingface\hub\models--nlptown--bert-base-multilingual-uncased-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mod

In [13]:
df_sentiment['puntaje']=resultados.apply(lambda x: x["score"])

C:\Users\SANTI\AppData\Local\Temp\ipykernel_14436\1764502434.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['puntaje']=resultados.apply(lambda x: x["score"])


In [14]:
df_sentiment['puntaje']=df_sentiment['puntaje'].apply(lambda x: 2*x-1)

C:\Users\SANTI\AppData\Local\Temp\ipykernel_14436\914804743.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sentiment['puntaje']=df_sentiment['puntaje'].apply(lambda x: 2*x-1)


In [15]:
df_sentiment.head(10)

,fecha,titulo,texto_procesado,puntaje
0,01/05/2025,"CZ: Para tener libertad económica, hay que ten...",cz ten libert econom ten libert expresion,-0.267648
1,01/05/2025,Bitcoin se recuperó tras datos bajistas del PI...,bitcoin recuper tras dat bajist pib eeuu mient...,-0.048909
2,01/05/2025,Qué siete valores del Ibex han marcado récord ...,siet valor ibex marc record abril,-0.358583
3,01/05/2025,Una primera vuelta en medio de la furia,primer vuelt medi furi,-0.355365
4,01/05/2025,Tether reporta 1.000 millones de dólares en ga...,teth report millon dolar gananci oper prim tri...,0.135429
5,01/05/2025,"La historia del ""desacoplamiento"" de las cript...",histori desacopl criptomoned termin accion sig...,0.317125
6,30/04/2025,3 gráficos de Ethereum mostraron una señal que...,grafic ethereum mostr señal vio sub,-0.502416
7,30/04/2025,Stablecoins alcanzarían los 2 billones de dóla...,stablecoins alcanz billon dolar capitaliz merc,-0.039940
8,30/04/2025,El PIB de EEUU alimenta preocupaciones de rece...,pib eeuu aliment preocup recesion bitcoin mant...,-0.565011
9,29/04/2025,"Trump, del júbilo al abismo",trump jubil abism,-0.487561


In [16]:
df_sentiment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   fecha            45 non-null     object 
 1   titulo           45 non-null     object 
 2   texto_procesado  45 non-null     object 
 3   puntaje          45 non-null     float64
dtypes: float64(1), object(3)
memory usage: 1.5+ KB


In [17]:
df_sentiment['fecha'].value_counts()

fecha
01/05/2025    6
30/04/2025    3
29/04/2025    3
28/04/2025    3
27/04/2025    3
26/04/2025    3
25/04/2025    3
24/04/2025    3
23/04/2025    3
22/04/2025    3
21/04/2025    3
20/04/2025    3
19/04/2025    3
18/04/2025    3
Name: count, dtype: int64

# Dataframe para el modelo

In [18]:
# Guardar el dataframe en la base de datos
import sqlite3
from datetime import datetime
import pandas as pd
import os

def save_to_db(df, db_path, table_name):
    # Conectar a la base de datos SQLite (o crearla si no existe)
    conn = sqlite3.connect(db_path)
    
    # Guardar el DataFrame en la tabla especificada
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    
    # Cerrar la conexión
    conn.close()

# Guardar el DataFrame en la base de datos
save_to_db(df_sentiment, '../Datos/data.db', 'Noticias_Sentimiento')